In [40]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Load your dataset
file_path = "/Users/lalitaneeharikavajjhala/Desktop/Research credits /Data/preprocessed_data.csv"
df = pd.read_csv(file_path)

df.head


<bound method NDFrame.head of                                tag  \
0                   ['obligation']   
1                   ['obligation']   
2                   ['obligation']   
3                   ['obligation']   
4                   ['obligation']   
..                             ...   
942  ['prohibition', 'obligation']   
943                 ['permission']   
944                 ['permission']   
945                ['prohibition']   
946                ['prohibition']   

                                              sentence  
0    we will issue a certificate of completion for ...  
1    elephant talk bear the risk of and shall indem...  
2    subject to the term and condition of this agre...  
3    ediets shall ensure that the ediets content co...  
4    auriemma will participate in one recording ses...  
..                                                 ...  
942  a party shall not unreasonably withhold or del...  
943  except a otherwise provided if performance her...  


In [41]:

# Tokenize the sentences in your dataset
sentences = []
for sentence in df['sentence']:
    tokens = word_tokenize(sentence)
    sentences.append(tokens)

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences, min_count=1, vector_size=100, window=5)

# Save the trained model to a file
word2vec_model.save("/Users/lalitaneeharikavajjhala/Desktop/Research credits /Models/word2vec_model.bin")


In [47]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from gensim.models import Word2Vec

# Loading preprocessed dataset
file_path = "/Users/lalitaneeharikavajjhala/Desktop/Research credits /Data/preprocessed_data.csv"
df = pd.read_csv(file_path)

# Converting tags from strings to lists
df['tag'] = df['tag'].apply(lambda x: literal_eval(x))

# Encode tags 'y'
y = df['tag']
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(y)

# Tokenize sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['sentence'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['sentence'])

# Padding sequences
maxlen = 200
X = pad_sequences(sequences, maxlen=maxlen)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['sentence'], vector_size=100, window=5, min_count=1, workers=4)

# Prepare embedding matrix
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True, stratify=y)

# Define CNN model with additional layers and hyperparameters
num_filters = 256
kernel_sizes = [3, 4, 5]

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 3, activation='relu', padding='same'))
model.add(Conv1D(num_filters, 3, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Define callbacks
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4)
]

# Train the model with more epochs and batch size
history = model.fit(X_train, y_train, epochs=30, batch_size=128, callbacks=callbacks, validation_split=0.2)

# Evaluate model
metrics = model.evaluate(X_test, y_test)
print("Test Loss: {:.4f}, Test Accuracy: {:.4f}".format(metrics[0], metrics[1]))

# Predicting
x = ["The confidentiality obligations contained in this section XI shall not apply to the extent that the receiving Party (the 'Recipient') is required (a) to disclose information by law, order or regulation of a governmental agency or a court of competent jurisdiction , or (b) to disclose information to any governmental agency for purposes of obtaining approval to test or market a Product , provided in either case that the Recipient shall provide written notice thereof to the other Party and sufficient opportunity to object to any such disclosure or to request confidential treatment thereof."]
xt = tokenizer.texts_to_sequences(x)
xt = pad_sequences(xt, maxlen=maxlen)
prediction = model.predict(xt)

probas = (prediction > 0.5).astype(int)
tags = multilabel.inverse_transform(probas)

print("Predicted Tags:", tags)


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_31 (Embedding)    (None, 200, 100)          292700    
                                                                 
 conv1d_40 (Conv1D)          (None, 200, 256)          77056     
                                                                 
 conv1d_41 (Conv1D)          (None, 200, 256)          196864    
                                                                 
 global_max_pooling1d_23 (G  (None, 256)               0         
 lobalMaxPooling1D)                                              
                                                                 
 dense_37 (Dense)            (None, 512)               131584    
                                                                 
 dropout_26 (Dropout)        (None, 512)               0         
                                                     